<a href="https://colab.research.google.com/github/polymathkids/PythonSandbox/blob/main/Deep_learning_Predicting_CTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Perceptrons
# input features are standardized with inputs summed through weights.
# the output goes through an activation function
# the ourput is put through.a unit step function to covert it into the predicted class
# one hidden layer for a perceptron

# clf = MLPClassifer()
#  The main parameters are activation (the type of activation function)
# alpha (regularization constant), hidden_layer_sizes (representing the hidden layers),
# learning rate (which determines how quickly the network tunes weights based on feedback from training)
# and max_iter (the number of iterations of training).
# The hidden layer sizes is an object such that the ith element represents
# the number of neurons in the ith hidden layer.
# So in this example, the model has 100 hidden units within 1 hidden layer.
# hyperparameters: activiation = 'relu', alpha = 0.0001, learning_rate = 'constant', max_iter = 200, hidden_layer_sizes = (100,)

#inputs must be standardized to help with convergence with something like sklearn's StandardScaler
# X = StandardScaler(),fit_transform(k)

In [20]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.datasets import load_digits
image_data = load_digits()
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, roc_curve, auc, roc_auc_score, fbeta_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import random

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [5]:
# Define X and y
X, y = image_data.data, image_data.target

# Scale features and split into training and testing
X_scaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
  X_scaled, y, test_size = .2, random_state = 0)

# Create classifier, train and evaluate accuracy 
clf = MLPClassifier()
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(accuracy_score(y_test, y_pred))

0.975


In [4]:
#import data from Ad-CTR sample (Kagggle)

dtype={'id': np.dtype(int),
    'click': np.dtype(int),
    'hour': str,
    'C1': np.dtype(str),
    'banner_pos': np.dtype(str),
    'site_id': np.dtype(str),
    'site_domain': np.dtype(str), 
    'site_category': np.dtype(str),
    'app_id': np.dtype(str),
    'app_domain': np.dtype(str),
    'app_category': np.dtype(str),
    'device_id': np.dtype(str),
    'device_ip': np.dtype(str),
    'device_model': np.dtype(str),
    'device_type': np.dtype(str),
    'device_conn_type': np.dtype(str),
    'C14': np.dtype(str),
    'C15': np.dtype(str),
    'C16': np.dtype(str),
    'C17': np.dtype(str),
    'C18': np.dtype(str),
    'C19': np.dtype(str),
    'C20': np.dtype(str),
    'C21':np.dtype(str)
      }
num_records = 40428967
sample_size = 500000
skip_values = sorted(random.sample(range(1,num_records), num_records - sample_size))
parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')

CTR_ad = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ColabData/avazu-ctr-prediction/train.gz", parse_dates=['hour'], date_parser=parse_date, dtype=dtype, skiprows=skip_values)

# extract hour of day
CTR_ad['hour_of_day'] = CTR_ad['hour'].dt.hour
#Guessing on numerical column function names:
CTR_ad.rename(columns = {'C1':'search_engine_type'}, inplace = True) #C1 is search_engine_type
CTR_ad.rename(columns = {'C14':'product_type'}, inplace = True) #C14 is product_type
CTR_ad.rename(columns = {'C15':'advertiser_type'}, inplace = True) #C15 is advertiser_type

# Get categorical columns
categorical_cols = CTR_ad.select_dtypes(
  include = ['object']).columns.tolist()
# Iterate over categorical columns and apply hash function
for col in categorical_cols:
	CTR_ad[col] = CTR_ad[col].apply(lambda x: hash(x))
 
feature_list = ["search_engine_type", "product_type", "advertiser_type"]
# Define new features as counts
new_feature_list = ['device_id', 'site_id'] + feature_list
for new_feature in new_feature_list:
  CTR_ad[new_feature + '_count'] = CTR_ad.groupby(
    new_feature)['click'].transform("count")
# Get non-categorical columns, with a filter
num_CTR_ad = CTR_ad.select_dtypes(include=['int', 'float'])
filter_cols = ['click', 'banner_pos', 'device_type',
               'search_engine_type', 'product_type', 'advertiser_type']
new_CTR_ad = num_CTR_ad[num_CTR_ad.columns[~num_CTR_ad.columns.isin(filter_cols)]]
num_cols = new_CTR_ad.columns

# Define X and y
y = CTR_ad.click
X = CTR_ad[num_cols]

y.to_csv('/content/drive/MyDrive/Colab Notebooks/ColabData/avazu-ctr-prediction/CTR_ad_y_click.csv')
X.to_csv('/content/drive/MyDrive/Colab Notebooks/ColabData/avazu-ctr-prediction/CTR_ad_X_pre_processed.csv')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.


In [5]:
# Scale features and split into training and testing
X_scaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
  X_scaled, y, test_size = .2, random_state = 0)

  # Create classifier and produce predictions
clf = MLPClassifier(hidden_layer_sizes = (8, ), max_iter = 100)
y_score = clf.fit(X_train, y_train).predict_proba(X_test) 
y_pred = clf.fit(X_train, y_train).predict(X_test) 

# Get accuracy and AUC of ROC curve 
fpr, tpr, thresholds = roc_curve(y_test, y_score[:, 1])
roc_auc = auc(fpr, tpr)
print("Accuracy: %s" %(accuracy_score(y_test, y_pred)))
print("ROC of AUC curve: %s" %(roc_auc))

Accuracy: 0.83243
ROC of AUC curve: 0.6857423199659023


In [12]:
# How to better tune the hyperparameters
# Use GridSearchCV to pass in options for tuning
# param_grid = {'mat_iter': [10, 20], 'hidden_layer_sizes': [8, ), (16, )]}
# clf = GridSearchCV(estimator = MLPClassifier(), param_grid = param_grid, n_jobs = 4)
# n_jobs = numbers of jobs that be run in parallell to speed up computation
# print(clf.best_score_)
# print(clf.best_estimator_)

#batch size and number of epochs are other potential hyperparameters,
#  the initilization of eights can also vary and affect results 
# usually Keras or TensorFlow are used as there are pre-tuned models that can be used to initialize

In [15]:
# Loop over various max_iter configurations
max_iter_list = [10, 20, 30]
for max_iter in max_iter_list:
	clf = MLPClassifier(hidden_layer_sizes = (4, ), 
                        max_iter = max_iter, random_state = 0)
   	# Extract relevant predictions
	y_score = clf.fit(X_train, y_train).predict_proba(X_test)
	y_pred = clf.fit(X_train, y_train).predict(X_test)

	# Get ROC curve metrics
	print("Accuracy for max_iter = %s: %s" %(
      max_iter, accuracy_score(y_test, y_pred)))
	print("AUC for max_iter = %s: %s" %(
      max_iter, roc_auc_score(y_test, y_score[:, 1])))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Accuracy for max_iter = 10: 0.830581
AUC for max_iter = 10: 0.6753220711942387
Accuracy for max_iter = 20: 0.830597
AUC for max_iter = 20: 0.6751495756116213
Accuracy for max_iter = 30: 0.830597
AUC for max_iter = 30: 0.6751495756116213


In [16]:
# Create and loop over various hidden_layer_sizes configurations
hidden_layer_sizes_list = [(4, ), (8, ), (16, )]
for hidden_layer_sizes in hidden_layer_sizes_list:
	clf = MLPClassifier(hidden_layer_sizes = hidden_layer_sizes, 
                        max_iter = 10, random_state = 0)
   	# Extract relevant predictions
	y_score = clf.fit(X_train, y_train).predict_proba(X_test)
	y_pred = clf.fit(X_train, y_train).predict(X_test)

	# Get ROC curve metrics
	print("Accuracy for hidden_layer_sizes = %s: %s" %(
      hidden_layer_sizes, accuracy_score(y_test, y_pred)))
	print("AUC for hidden_layer_sizes = %s: %s" %(
      hidden_layer_sizes, roc_auc_score(y_test, y_score[:, 1])))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Accuracy for hidden_layer_sizes = (4,): 0.830581
AUC for hidden_layer_sizes = (4,): 0.6753220711942387


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Accuracy for hidden_layer_sizes = (8,): 0.829077
AUC for hidden_layer_sizes = (8,): 0.6882671115510715


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Accuracy for hidden_layer_sizes = (16,): 0.831322
AUC for hidden_layer_sizes = (16,): 0.6994303025566845


In [6]:
# MLP Grid Search
# Create list of hyperparameters 
max_iter = [10, 20]
hidden_layer_sizes = [(8, ), (16, )]
param_grid = {'max_iter': max_iter, 'hidden_layer_sizes': hidden_layer_sizes}

# Use Grid search CV to find best parameters using 4 jobs
mlp = MLPClassifier()
clf = GridSearchCV(estimator = mlp, param_grid = param_grid, 
           scoring = 'roc_auc', n_jobs = 4)
clf.fit(X_train, y_train)
print("Best Score: ")
print(clf.best_score_)
print("Best Estimator: ")
print(clf.best_estimator_)

Best Score: 
0.690687969229761
Best Estimator: 
MLPClassifier(hidden_layer_sizes=(16,), max_iter=20)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [7]:
# Precision and recall
# Precision - ROI on ad spend through clicks
# if precision is (low) - little tangible ROI on clicks
# Recall - how well you are targeting relevant audience
# if recall is low- it meas you have missed out on opportunities on ROI
# ex: the company is not showing as many relevant ads as it could have done

#Since precision is more tangible, the metrics are weighted to show preference for precision using the F-beta Score
# F-beta score is a weighted harmonic mean between precision and recall
# (1+ B^2) * ((precision * recall)/(B^2*precision) + recall))
# if B = 1, they are weighted equally. If B >1 then precision is made larger and weighted less
# if B is between 0 and 1, then precision is made ssmaller and therefore weighted more

# sklearn: fbeta_score(y_true, y_pred, beta)
# another important evaluation metric is the AUC or ROC curve vs precision
# roc_auc = roc_auc_score(y_test, y_score[:, 1])

#remember-
#fpr = 1-tn/(tn + fp) false positive rate
#precision = tp/(tp + fp)
# note if you have an imbalanced dataset- fpr can be low when precision is also low
# ex: assume tn = 100, tp = 10 and fp = 10
# then fpr = 0.091 (low) and precision = 0.5 (coin flip, not good)

In [11]:
# Set up MLP classifier, train and predict
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size = .2, random_state = 0)
clf = MLPClassifier(hidden_layer_sizes = (16, ), 
                    max_iter = 50, random_state = 0)
y_pred = clf.fit(X_train, y_train).predict(X_test) 

# Evaluate precision and recall
prec = precision_score(y_test, y_pred, average = 'weighted')
recall = recall_score(y_test, y_pred, average = 'weighted')
fbeta = fbeta_score(y_test, y_pred, beta  = 0.5, average = 'weighted')
print("Precision: %s, Recall: %s, F-beta score: %s" %(prec, recall, fbeta))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Precision: 0.7464388712506983, Recall: 0.7578, F-beta score: 0.7485576683297585


In [12]:
# Get precision and total ROI
prec = precision_score(y_test, y_pred, average = 'weighted')
r = 0.2
cost = 0.05 
roi = prec * r / cost

# Get AUC
roc_auc = roc_auc_score(y_test, y_score[:, 1])

print("Total ROI: %s, Precision: %s, AUC of ROC curve: %s" %(
  roi, prec, roc_auc))

#Note the ROI was > 1 and the precision and ROC of the AUC are both > 0.65, suggesting this not a case of low precision and high AUC.

Total ROI: 2.985755485002793, Precision: 0.7464388712506983, AUC of ROC curve: 0.6857423199659023


In [15]:
# Model implementation
# machine learning vs Deep learning
# Compare MLP and Random Forest models using a confusion matris:
# assumes workspace contains training and testing splits
# for X and y as X_train, X_test for X and y_train, y_test for y respectively.
# Remember, X contains our engineered features with user, device, and site details
# whereas y contains the target (whether the ad was clicked).
# Assumes X has already been scaled using a StandardScaler()


# Create the list of models in the order below
names = ['Random Forest', 'Multi-Layer Perceptron']
classifiers = [RandomForestClassifier(), 
               MLPClassifier(hidden_layer_sizes = (10, ),
                             max_iter = 40)]

# Produce a confusion matrix for all classifiers
for name, classifier in zip(names, classifiers):
  print("Evaluating classifier: %s" %(name))
  classifier.fit(X_train, y_train)
  y_pred = classifier.predict(X_test)
  conf_matrix = confusion_matrix(y_test, y_pred)
  print(conf_matrix)

Evaluating classifier: Random Forest
[[79576  3633]
 [14487  2304]]
Evaluating classifier: Multi-Layer Perceptron
[[74446  8763]
 [14898  1893]]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [21]:
#Evaluating precision and ROI
# Create list of classifiers
names = ['Logistic Regression',  'Decision Tree',
         'Random Forest', 'Multi-Layer Perceptron']
clfs = [LogisticRegression(), 
        DecisionTreeClassifier(), RandomForestClassifier(), 
        MLPClassifier(hidden_layer_sizes = (5, ), max_iter = 40)]

# Fit each classifier and evaluate AUC of ROC curve 
r, cost = 0.2, 0.05 
for name, classifier in zip(names, clfs):
  classifier.fit(X_train, y_train)
  y_score = classifier.predict_proba(X_test)
  y_pred = classifier.predict(X_test) 
  prec = precision_score(y_test, y_pred, average = 'weighted')
  print("Precision for %s: %s " %(name, prec))
  roi = prec * r / cost
  print("ROI for %s: %s " %(name, roi))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision for Logistic Regression: 0.6923737681000001 
ROI for Logistic Regression: 2.7694950724000003 
Precision for Decision Tree: 0.7552130595770128 
ROI for Decision Tree: 3.0208522383080516 
Precision for Random Forest: 0.7687952567749987 
ROI for Random Forest: 3.075181027099995 
Precision for Multi-Layer Perceptron: 0.7158655616755539 
ROI for Multi-Layer Perceptron: 2.8634622467022157 


In [24]:
# Create classifiers
clfs = [LogisticRegression(), DecisionTreeClassifier(), RandomForestClassifier(), 
        MLPClassifier(hidden_layer_sizes = (10, ), max_iter = 50)]

def print_estimator_name(estimator):
        return estimator.__class__.__name__

# Produce all evaluation metrics for each classifier
for clf in clfs:
  print("Evaluating classifier: %s" %(print_estimator_name(clf)))
  y_score = clf.fit(X_train, y_train).predict_proba(X_test)
  y_pred = clf.fit(X_train, y_train).predict(X_test)
  prec = precision_score(y_test, y_pred, average = 'weighted')
  recall = recall_score(y_test, y_pred, average = 'weighted')
  fbeta = fbeta_score(y_test, y_pred, beta = 0.5, average = 'weighted')
  roc_auc = roc_auc_score(y_test, y_score[:, 1])
  print("Precision: %s: Recall: %s, F-beta score: %s, AUC of ROC curve: %s" 
        %(prec, recall, fbeta, roc_auc))

Evaluating classifier: LogisticRegression


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGIS

Precision: 0.6923737681000001: Recall: 0.83209, F-beta score: 0.716433021839411, AUC of ROC curve: 0.5
Evaluating classifier: DecisionTreeClassifier
Precision: 0.7558978104019491: Recall: 0.74912, F-beta score: 0.7544874154525861, AUC of ROC curve: 0.5639431562711649
Evaluating classifier: RandomForestClassifier
Precision: 0.7696796165787445: Recall: 0.81874, F-beta score: 0.7690136893973429, AUC of ROC curve: 0.6991665976929342
Evaluating classifier: MLPClassifier


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Precision: 0.7523924627642931: Recall: 0.81011, F-beta score: 0.7547763268798078, AUC of ROC curve: 0.5000786422583158
